# NCME 2023 Data Science Upskill Workshop - Session 4
# Accelerating Python with numba, multiprocessing, and dask
#### Questions contact: jhao@ets.org

In [ ]:
from numba import jit, njit
import numpy as np
import time
import graphviz

## 1. Jit from numba

In [ ]:
def naive_sum(n):
    s = 0
    for i in range(n):
        s = i+1
    return s

# using jit. 
@jit
def naive_sum_jit(n):
    s = 0
    for i in range(n):
        s = i+1
    return s

# Note that the @jit is the decorator function. It takes another function as its argument, 
# and returns yet another function. Decorators can be extremely useful as they allow the extension of 
# an existing function, without any modification to the original function source code.
# the above function means naive_sum_jit = jit(naive_sum_jit)


# njit is the jit(nonpython=True)
@njit  
def naive_sum_njit(n):
    s = 0
    for i in range(n):
        s = i+1
    return s


In [ ]:
n = 10000000

In [ ]:
%%timeit
naive_sum(n)

In [ ]:
%%timeit 
naive_sum_jit(n)

In [ ]:
%%timeit
naive_sum_njit(n)

## 2. Universal function (ufunc) and vectorization using numba

In [ ]:
from numba import vectorize

In [ ]:
# direct loop
def naive_array_sum_loop(a,b):
    c = np.zeros(n)
    for i in range(n):
        c[i] = a[i]+b[i]
    return c

# using jit to accelerate
@njit
def naive_array_sum_njit(a,b):
    c = np.zeros(n)
    for i in range(n):
        c[i] = a[i]+b[i]
    return c

# creating ufunc using vectorize from numba
@vectorize
def naive_array_sum_ufunc(a,b):
    c = a + b
    return c

In [ ]:
a = np.arange(n)
b = np.arange(n)

In [ ]:
a

In [ ]:
b

In [ ]:
print(type(naive_array_sum_loop))
%timeit naive_array_sum_loop(a,b)

In [ ]:
print(type(naive_array_sum_njit))  
%timeit naive_array_sum_njit(a,b)

In [ ]:
print(type(naive_array_sum_ufunc))
%timeit naive_array_sum_ufunc(a,b)

## 3. Accelerating pandas with numba 

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame([[2, 4]] * 10000, columns=['A', 'B'])

In [ ]:
df.shape

In [ ]:
df

In [ ]:
# simple square
def get_square(x):
    return x**2

# jit version
@njit
def get_square_njit(x):
    return x**2

# ufunct using vectorize
@vectorize
def get_square_vectorize(x):
    return x**2

In [ ]:
%%time
# direct loop
for i in range(df.shape[0]):
    df.at[i,'square_A'] = get_square(df.A.iloc[i])

In [ ]:
%%time
# using apply
df['square_A'] = df.A.apply(get_square)


In [ ]:
%%time
# using apply with njit
df['square_A'] = df.A.apply(get_square_njit)

In [ ]:
%%time
# using apply with ufunc
df['square_A'] = df.A.apply(get_square_vectorize)

In [ ]:
%%time
# using vectorization
df['square_A'] = get_square(df.A)

In [ ]:
%%time
#using vectorization a different way
df['square_A'] = get_square_vectorize(df.A)

## 5. Parallelization

### 5.0 Parallelization using multiprocessing

In [ ]:
from multiprocessing import Pool #package comes with python distribution
from myfunctions import slow_add 

In [ ]:
#more data
data = [1, 2, 3, 4, 5, 6]

In [ ]:
%%time
for x in data:
    slow_add(x,1)

In [ ]:
%%time
# multiprocessing approach
args = list(zip(data,[1]*6))
with Pool(6) as p:
    result = p.starmap(slow_add,args)

In [ ]:
result

### 5.1 Parallelization using Dask delayed

In [ ]:
%%time
# This takes three seconds to run because we call each

x = slow_add(1, 2)
y = slow_add(2, 3)
z = slow_add(x, y)

print("z is", z)

In [ ]:
from dask import delayed

In [ ]:
# make a delayed function
@delayed
def delayed_slow_add(x, y):
    time.sleep(1)
    return x + y

In [ ]:
%%time
# This runs immediately, all it does is build a graph

x = delayed_slow_add(1, 2)
y = delayed_slow_add(2, 3)
z = delayed_slow_add(x, y)
z

In [ ]:
# visualize the task graph
z.visualize()

In [ ]:
%%time
# This actually runs our computation
# using a local thread pool

z.compute()

In [ ]:
%%time
# an alternative way to use the delayed on existing function

x = delayed(slow_add)(1, 2)
y = delayed(slow_add)(2, 3)
z = delayed(slow_add)(x, y)
z

In [ ]:
%%time
# This actually runs our computation
# using a local thread pool

z.compute()

In [ ]:
%%time
# Consider a list using convential loop

results = []

for x in data:
    y = slow_add(x, 1)
    results.append(y)
    
total = sum(results)
print("total is", total)

In [ ]:
# now let's use the delayed version

results = []

for x in data:
    y = delayed_slow_add(x, 1)
    results.append(y)
    
total = delayed(sum)(results)

# Let's see what type of thing total is
print("Printing total: ", total)

In [ ]:
total.visualize()

In [ ]:
%%time

# Compuing time
result = total.compute()
print("Printing result from computing total:", result)

In [ ]:
# what if I have more processes than the number of my CPU cores?
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [ ]:
%%time
# Consider a list using convential loop

results = []

for x in data:
    y = slow_add(x, 1)
    results.append(y)
    
total = sum(results)
print("total is", total)

In [ ]:
# now let's use the delayed version

results = []

for x in data:
    y = delayed_slow_add(x, 1)
    results.append(y)
    
total = delayed(sum)(results)

# Let's see what type of thing total is
print("Printing total: ", total)

In [ ]:
total.visualize()

In [ ]:
%%time

# Compuing ...
result = total.compute()
print("Total:", result)

### 5.2 Scheduling - single machine

#### 5.2.1 Multiple local processes - cores of CPU

In [ ]:
# find out how many logic 
import os
os.cpu_count()

In [ ]:
total.visualize()

In [ ]:
# use 12 cores
%time result = total.compute(scheduler='processes', num_workers = 5)

In [ ]:
#use 24 cores
%time result = total.compute(scheduler='processes', num_workers = 20)

#### 5.2.2 Multiple threads

In [ ]:
# Single thread ...
%time result = total.compute(scheduler='synchronous')

In [ ]:
# Using more threads
%time result = total.compute(scheduler='threads', num_workers=20)

In [ ]:
%time result = total.compute(scheduler='threads', num_workers=5)

### --- The End ----